In [9]:
pwd = '/home/zjy/project/MetaIM/deepIM/data_deepIM/'

In [10]:
import pickle
import argparse

device='cpu'

parser = argparse.ArgumentParser(description="GenIM")
datasets = ['jazz', 'cora_ml', 'power_grid', 'netscience', 'random5']
parser.add_argument("-d", "--dataset", default="cora_ml", type=str,
                    help="one of: {}".format(", ".join(sorted(datasets))))
diffusion = ['IC', 'LT', 'SIS']
parser.add_argument("-dm", "--diffusion_model", default="SIS", type=str,
                    help="one of: {}".format(", ".join(sorted(diffusion))))
seed_rate = [1, 5, 10, 20]
parser.add_argument("-sp", "--seed_rate", default=1, type=int,
                    help="one of: {}".format(", ".join(str(sorted(seed_rate)))))
mode = ['Normal', 'Budget Constraint']
parser.add_argument("-m", "--mode", default="normal", type=str,
                    help="one of: {}".format(", ".join(sorted(mode))))
args = parser.parse_args(args=[])


with open(pwd+ args.dataset + '_mean_' + args.diffusion_model + str(50) + '.SG', 'rb') as f:
    graph = pickle.load(f)

/tmp/ipykernel_981237/1388658084.py:23: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  graph = pickle.load(f)


In [14]:
graph['inverse_pairs'][0][:,0].sum()

tensor(140.)

In [2]:
import numpy as np
import networkx as nx
import ndlib.models.ModelConfig as mc
import ndlib.models.epidemics as ep

def diffusion_evaluation(adj_matrix, seed, diffusion='LT'):

    total_infect = 0
    G = nx.from_scipy_sparse_array(adj_matrix)
    inflect_vec = np.zeros(len(G.nodes()))
    for i in range(10):
        
        if diffusion == 'LT':
            model = ep.ThresholdModel(G)
            config = mc.Configuration()
            for n in G.nodes():
                config.add_node_configuration("threshold", n, 0.5)
        elif diffusion == 'IC':
            model = ep.IndependentCascadesModel(G)
            config = mc.Configuration()
            for e in G.edges():
                config.add_edge_configuration("threshold", e, 1/nx.degree(G)[e[1]])
        elif diffusion == 'SIS':
            model = ep.SISModel(G)
            config = mc.Configuration()
            config.add_model_parameter('beta', 0.001)
            config.add_model_parameter('lambda', 0.001)
        else:
            raise ValueError('Only IC, LT and SIS are supported.')

        config.add_model_initial_configuration("Infected", seed)

        model.set_initial_status(config)

        iterations = model.iteration_bunch(100)

        node_status = iterations[0]['status']

        seed_vec = np.array(list(node_status.values()))

        for j in range(1, len(iterations)):
            node_status.update(iterations[j]['status'])

        inf_vec = np.array(list(node_status.values()))
        inf_vec[inf_vec == 2] = 1
        
        inflect_vec += inf_vec

        # total_infect += inf_vec.sum()
    
    inflect_vec[inflect_vec > 0] = 1
    
    return inflect_vec

In [5]:
import torch
input = torch.tensor(graph['inverse_pairs'][0][:,0])
seed = (input > 0).nonzero().flatten().detach().numpy()
seed

/var/folders/9b/51r_nlz93n7__jn387b614040000gn/T/ipykernel_81968/4003716307.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input = torch.tensor(graph['inverse_pairs'][0][:,0])


array([   5,   17,   33,   34,   35,   45,   47,   49,   57,   59,   60,
         66,   82,   84,   86,   87,   93,   94,   96,  102,  103,  106,
        107,  119,  124,  129,  132,  140,  169,  188,  189,  234,  236,
        241,  249,  264,  315,  347,  413,  433,  466,  499,  509,  525,
        554,  555,  557,  562,  565,  589,  602,  611,  614,  645,  693,
        730,  851,  852,  856, 1021, 1049, 1059, 1060, 1063, 1101, 1109,
       1146, 1152, 1185, 1190, 1199, 1261, 1268, 1278, 1289, 1296, 1341,
       1385, 1391, 1393, 1399, 1416, 1439, 1446, 1450, 1563, 1581, 1582,
       1702, 1724, 1763, 1775, 1787, 1848, 1902, 1933, 1946, 1955, 1969,
       1973, 2036, 2100, 2108, 2111, 2120, 2122, 2130, 2140, 2171, 2197,
       2210, 2240, 2301, 2325, 2344, 2347, 2349, 2374, 2390, 2401, 2402,
       2407, 2470, 2483, 2504, 2545, 2550, 2568, 2573, 2575, 2677, 2698,
       2700, 2722, 2735, 2736, 2742, 2751, 2769, 2773])

In [6]:
adj = graph['adj']
inflect_vec = diffusion_evaluation(adj, seed, diffusion = args.diffusion_model)
inflect_vec.sum()

1396.0

In [7]:
inflect_vec_t = np.zeros((len(inflect_vec),len(inflect_vec)))
inflect_vec_t

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [8]:
import concurrent.futures

# 定义任务函数
def evaluate_diffusion(adj_matrix, seed, diffusion_model):
    return diffusion_evaluation(adj_matrix, seed, diffusion_model)

# 创建线程池
with concurrent.futures.ThreadPoolExecutor() as executor:
    # 提交任务并获取结果
    future_to_index = {executor.submit(evaluate_diffusion, adj, [i], args.diffusion_model): i for i in range(len(inflect_vec))}
    for future in concurrent.futures.as_completed(future_to_index):
        index = future_to_index[future]
        inflect_vec_t[index] = future.result()

# 输出结果
print(inflect_vec_t)


# for i in range(len(inflect_vec)):
#     inflect_vec_t[i] = diffusion_evaluation(adj, [i], diffusion = args.diffusion_model)
# inflect_vec_t

In [ ]:
# 将数组保存到本地文件
np.save(f'cora_{args.diffusion_model}_50.npy', inflect_vec_t)

In [2]:
import numpy as np

# 读取 .npy 文件
data = np.load(f'cora_{args.diffusion_model}_50_e.npy')
data


NameError: name 'args' is not defined

In [22]:
inflect_vec_T = np.zeros(len(inflect_vec))
for i in range(len(seed)):
    inflect_vec_T += inflect_vec_t[seed[i]]

inflect_vec_T[inflect_vec_T > 0] = 1
inflect_vec_T.sum()

487.0

In [6]:
graph_out = graph
graph_out

{'adj': <2810x2810 sparse matrix of type '<class 'numpy.float32'>'
 	with 15962 stored elements in Compressed Sparse Row format>,
 'inverse_pairs': tensor([[[0., 0.],
          [0., 0.],
          [0., 0.],
          ...,
          [0., 0.],
          [0., 0.],
          [0., 0.]],
 
         [[0., 1.],
          [0., 0.],
          [0., 0.],
          ...,
          [0., 0.],
          [0., 0.],
          [0., 0.]],
 
         [[0., 0.],
          [0., 0.],
          [0., 0.],
          ...,
          [0., 0.],
          [0., 0.],
          [0., 0.]],
 
         ...,
 
         [[0., 0.],
          [0., 1.],
          [0., 0.],
          ...,
          [0., 1.],
          [0., 0.],
          [0., 0.]],
 
         [[0., 0.],
          [0., 0.],
          [0., 0.],
          ...,
          [0., 0.],
          [0., 0.],
          [0., 0.]],
 
         [[0., 0.],
          [0., 0.],
          [0., 0.],
          ...,
          [0., 0.],
          [0., 1.],
          [0., 0.]]])}

In [35]:
adj = graph['adj']
for i in range(graph['inverse_pairs'].shape[0]):
    input_t = torch.tensor(graph['inverse_pairs'][i][:,0])
    seed_t = (input_t > 0).nonzero().flatten().detach().numpy()
    inflect_vec_feat = np.zeros(len(inflect_vec))
    for j in range(len(seed_t)):
        inflect_vec_feat += inflect_vec_t[seed_t[j]]
    inflect_vec_feat[inflect_vec_feat > 0] = 1
    inflect_vec_label = diffusion_evaluation(adj, seed_t, diffusion = args.diffusion_model)
    graph_out['inverse_pairs'][i][:, 0] = torch.tensor(inflect_vec_feat)
    graph_out['inverse_pairs'][i][:, 1] = torch.tensor(inflect_vec_label)
    
graph_out

/var/folders/9b/51r_nlz93n7__jn387b614040000gn/T/ipykernel_23061/3219990898.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_t = torch.tensor(graph['inverse_pairs'][i][:,0])


{'adj': <2810x2810 sparse matrix of type '<class 'numpy.float32'>'
 	with 15962 stored elements in Compressed Sparse Row format>,
 'inverse_pairs': tensor([[[0., 0.],
          [0., 0.],
          [0., 0.],
          ...,
          [1., 1.],
          [0., 1.],
          [1., 1.]],
 
         [[0., 1.],
          [0., 1.],
          [0., 1.],
          ...,
          [1., 1.],
          [0., 1.],
          [0., 1.]],
 
         [[0., 1.],
          [0., 1.],
          [0., 1.],
          ...,
          [1., 1.],
          [0., 1.],
          [1., 1.]],
 
         ...,
 
         [[0., 0.],
          [0., 1.],
          [0., 1.],
          ...,
          [1., 1.],
          [0., 1.],
          [0., 0.]],
 
         [[0., 0.],
          [0., 1.],
          [0., 1.],
          ...,
          [1., 1.],
          [0., 1.],
          [0., 1.]],
 
         [[0., 1.],
          [0., 1.],
          [0., 1.],
          ...,
          [1., 1.],
          [0., 1.],
          [1., 1.]]])}

In [48]:
graph_out['inverse_pairs'] = graph_out['inverse_pairs'].numpy()

In [52]:
import pickle

# 保存字典到本地文件
with open('cora_SIS_50.SG', 'wb') as f:
    pickle.dump(graph_out, f)

### 开始训练

In [2]:
import pickle

with open('cora_LT_50.SG', 'rb') as f:
    graph = pickle.load(f)

In [3]:
import scipy.sparse as sp
from torch.utils.data import DataLoader
import numpy as np
import torch

def normalize_adj(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv_sqrt = np.power(rowsum, -0.5).flatten()
    r_inv_sqrt[np.isinf(r_inv_sqrt)] = 0.
    r_mat_inv_sqrt = sp.diags(r_inv_sqrt)
    return mx.dot(r_mat_inv_sqrt).transpose().dot(r_mat_inv_sqrt)

batch_size = 16
hidden_dim = 1024
latent_dim = 512

adj, inverse_pairs = graph['adj'], graph['inverse_pairs']
adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)
adj = normalize_adj(adj + sp.eye(adj.shape[0]))
adj = torch.Tensor(adj.toarray()).to_sparse()

train_set, test_set = torch.utils.data.random_split(inverse_pairs, 
                                                    [len(inverse_pairs)-batch_size, 
                                                     batch_size])
train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True, drop_last=False)
test_loader  = DataLoader(dataset=test_set,  batch_size=1, shuffle=False)

In [4]:
from main.model.gat import GAT, SpGAT
from torch.optim import Adam, SGD


forward_model = SpGAT(nfeat=1, 
                nhid=64, 
                nclass=1, 
                dropout=0.2, 
                nheads=4, 
                alpha=0.2)

optimizer = Adam([{'params': forward_model.parameters()}], 
                 lr=1e-3)

adj = adj.to(device)
forward_model = forward_model.to(device)
forward_model.train()

SpGAT(
  (attention_0): SpGraphAttentionLayer (1 -> 64)
  (attention_1): SpGraphAttentionLayer (1 -> 64)
  (attention_2): SpGraphAttentionLayer (1 -> 64)
  (attention_3): SpGraphAttentionLayer (1 -> 64)
  (attention1_0): SpGraphAttentionLayer (256 -> 64)
  (attention1_1): SpGraphAttentionLayer (256 -> 64)
  (attention1_2): SpGraphAttentionLayer (256 -> 64)
  (attention1_3): SpGraphAttentionLayer (256 -> 64)
  (out_att): SpGraphAttentionLayer (256 -> 1)
)

In [6]:
import time
import torch.nn.functional as F



for epoch in range(600):
    begin = time.time()
    total_overall = 0
    forward_loss = 0

    for batch_idx, data_pair in enumerate(train_loader):
        # input_pair = torch.cat((data_pair[:, :, 0], data_pair[:, :, 1]), 1).to(device)
        
        x = data_pair[:, :, 0].float().to(device)
        y = data_pair[:, :, 1].float().to(device)
        optimizer.zero_grad()
        
        loss = 0
        for i, x_i in enumerate(x):
            y_i = y[i]
            y_hat = forward_model(x_i.unsqueeze(-1), adj)
            # forward_loss = F.mse_loss(y_hat.squeeze(-1), y_i, reduction='sum')
            forward_loss = F.binary_cross_entropy(y_hat.squeeze(-1), y_i, reduction='sum')            
            loss += forward_loss    
        
        total_overall += loss.item()
        loss = loss/x.size(0)
        
        loss.backward()
        optimizer.step()
        for p in forward_model.parameters():
            p.data.clamp_(min=0)
        
        
    # 在测试集上进行评估
    test_loss = 0.0
    correct = 0
    correct_1 = 0
    # total = 0
    with torch.no_grad():
        for batch_idx, data_pair in enumerate(test_loader):
            x = data_pair[:, :, 0].float().to(device)
            y = data_pair[:, :, 1].float().to(device)
            
            correct_t = 0
            correct_t_1 = 0
            count = 0
            for i, x_i in enumerate(x):
                count += 1
                y_i = y[i]
            
                y_hat = forward_model(x_i.unsqueeze(-1), adj)
                # forward_loss = F.mse_loss(y_hat.squeeze(-1), y_i, reduction='sum')
                forward_loss = F.binary_cross_entropy(y_hat.squeeze(-1), y_i, reduction='sum')         
                test_loss += forward_loss
                
                # 将大于阈值的元素设置为 1，小于等于阈值的元素设置为 0
                threshold = 0.6
                y_pre = y_hat.squeeze(-1)
                
                filtered_y_hat = (y_pre > threshold).float()

                correct_t += ((filtered_y_hat == y_i).sum()/len(y_i))
                
                count_both_ones = torch.sum((filtered_y_hat == 1) & (y_i == 1))
                correct_t_1 += count_both_ones/y_i.sum()
                
            correct_t /= count
            correct_t_1 /= count
            correct += correct_t
            correct_1 += correct_t_1
            
    test_loss = test_loss / len(test_loader.dataset)
    test_accuracy = correct / len(test_loader.dataset)
    
    test_accuracy_1 = correct_1/ len(test_loader.dataset)
            
    end = time.time()
    print("Epoch: {}".format(epoch+1), 
          "\tTotal: {:.4f}".format(total_overall / len(train_set)),
          "\tTest_loss: {:.4f}".format(test_loss),
          "\tTest_accuracy: {:.4f}".format(test_accuracy),
          "\tTest_accuracy_1: {:.4f}".format(test_accuracy_1),
        #   "\tReconstruction Precision: {:.4f}".format(precision_re / len(train_set)),
        #   "\tReconstruction Recall: {:.4f}".format(recall_re / len(train_set)),
          "\tTime: {:.4f}".format(end - begin)
         )

Epoch: 1 	Total: 1499.0434 	Test_loss: 1424.5707 	Test_accuracy: 0.7877 	Test_accuracy_1: 0.8820 	Time: 5.9139
Epoch: 2 	Total: 1502.8730 	Test_loss: 1434.0422 	Test_accuracy: 0.7834 	Test_accuracy_1: 0.8795 	Time: 5.8210
Epoch: 3 	Total: 1500.1562 	Test_loss: 1435.6721 	Test_accuracy: 0.7844 	Test_accuracy_1: 0.8713 	Time: 5.6824
Epoch: 4 	Total: 1503.0981 	Test_loss: 1427.6118 	Test_accuracy: 0.7864 	Test_accuracy_1: 0.8793 	Time: 5.6809
Epoch: 5 	Total: 1500.2390 	Test_loss: 1427.3278 	Test_accuracy: 0.7844 	Test_accuracy_1: 0.8801 	Time: 5.7007
Epoch: 6 	Total: 1504.5316 	Test_loss: 1426.1654 	Test_accuracy: 0.7856 	Test_accuracy_1: 0.8805 	Time: 5.6905
Epoch: 7 	Total: 1498.1593 	Test_loss: 1430.8925 	Test_accuracy: 0.7825 	Test_accuracy_1: 0.8729 	Time: 5.6599
Epoch: 8 	Total: 1499.8725 	Test_loss: 1420.9655 	Test_accuracy: 0.7830 	Test_accuracy_1: 0.8890 	Time: 5.6602
Epoch: 9 	Total: 1503.3292 	Test_loss: 1428.4436 	Test_accuracy: 0.7848 	Test_accuracy_1: 0.8810 	Time: 5.6978
E

KeyboardInterrupt: 